In [1]:
!cp '/content/drive/Shareddrives/MEC - Correção textual/PLN/Dataset/Pontuação/ted-2012-03.tgz' ted-2012-03.tgz

!tar -xvzf ted-2012-03.tgz

2012-03/
2012-03/scores/
2012-03/._scores.html
2012-03/scores.html
2012-03/subeval_files/
2012-03/._subjeval.html
2012-03/subjeval.html
2012-03/texts/
2012-03/._texts-old.html
2012-03/texts-old.html
2012-03/._texts.html
2012-03/texts.html
2012-03/texts/ar/
2012-03/texts/de/
2012-03/texts/en/
2012-03/texts/nl/
2012-03/texts/pl/
2012-03/texts/pt-br/
2012-03/texts/ro/
2012-03/texts/ru/
2012-03/texts/sk/
2012-03/texts/sl/
2012-03/texts/tr/
2012-03/texts/zh/
2012-03/texts/zh/en/
2012-03/texts/zh/en/._.eval
2012-03/texts/zh/en/.eval
2012-03/texts/zh/en/._.info
2012-03/texts/zh/en/.info
2012-03/texts/zh/en/._zh-en.tgz
2012-03/texts/zh/en/zh-en.tgz
2012-03/texts/tr/en/
2012-03/texts/tr/en/._.eval
2012-03/texts/tr/en/.eval
2012-03/texts/tr/en/._.info
2012-03/texts/tr/en/.info
2012-03/texts/tr/en/._tr-en.tgz
2012-03/texts/tr/en/tr-en.tgz
2012-03/texts/sl/en/
2012-03/texts/sl/en/._.eval
2012-03/texts/sl/en/.eval
2012-03/texts/sl/en/._.info
2012-03/texts/sl/en/.info
2012-03/texts/sl/en/._sl-en.tgz

In [2]:
!tar -xvzf /content/2012-03/texts/pt-br/en/pt-br-en.tgz

pt-br-en/
pt-br-en/IWSLT12.TALK.tst2010.pt-br-en.en.xml
pt-br-en/train.en
pt-br-en/README
pt-br-en/IWSLT12.TALK.dev2010.pt-br-en.en.xml
pt-br-en/train.tags.pt-br-en.pt-br
pt-br-en/IWSLT12.TALK.tst2010.pt-br-en.pt-br.xml
pt-br-en/IWSLT12.TALK.dev2010.pt-br-en.pt-br.xml
pt-br-en/train.tags.pt-br-en.en


In [3]:
import os
import pandas as pd
from bs4 import BeautifulSoup as bs


try:
  os.makedirs('tedtalk2012-03/ptbr')
except OSError:
  pass 

for file in os.listdir("./pt-br-en"):
    if file.endswith(".pt-br") or file.endswith('.pt-br.xml'):
        with open(os.path.join("./pt-br-en", file)) as f:    
            bs_content = bs(f.read(), "lxml")

            
            transcript = bs_content.find_all('transcript')
            
            dts = file.split('.')[-4]
            if not transcript:

              segs = bs_content.find_all('seg')
              with open(f'./tedtalk2012-03/ptbr/tedtalk2012.{dts}', 'w') as f:
                  for seg in segs:
                    f.write(seg.get_text().strip()+'\n')
            else:

              with open(f'./tedtalk2012-03/ptbr/tedtalk2012.{dts}', 'w') as f:
                  for transp in transcript:
                    f.write(transp.get_text().strip()+'\n')


In [4]:
!cp '/content/drive/Shareddrives/MEC - Correção textual/PLN/Dataset/Corpus de Tipos Textuais Brasileiros/corpus-ttbr-v0.1.6-unbalanced.zip' corpus-ttbr-v0.1.6-unbalanced.zip
!unzip corpus-ttbr-v0.1.6-unbalanced.zip


A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: corpus-ttbr-v0.1.6-unbalanced/narrative/7323.txt  
  inflating: corpus-ttbr-v0.1.6-unbalanced/narrative/9714.txt  
  inflating: corpus-ttbr-v0.1.6-unbalanced/narrative/4060.txt  
  inflating: corpus-ttbr-v0.1.6-unbalanced/narrative/4752.txt  
  inflating: corpus-ttbr-v0.1.6-unbalanced/narrative/2186.txt  
  inflating: corpus-ttbr-v0.1.6-unbalanced/narrative/12195.txt  
  inflating: corpus-ttbr-v0.1.6-unbalanced/narrative/1517.txt  
  inflating: corpus-ttbr-v0.1.6-unbalanced/narrative/13639.txt  
  inflating: corpus-ttbr-v0.1.6-unbalanced/narrative/6721.txt  
  inflating: corpus-ttbr-v0.1.6-unbalanced/narrative/9640.txt  
  inflating: corpus-ttbr-v0.1.6-unbalanced/narrative/0363.txt  
  inflating: corpus-ttbr-v0.1.6-unbalanced/narrative/7078.txt  
  inflating: corpus-ttbr-v0.1.6-unbalanced/narrative/0567.txt  
  inflating: corpus-ttbr-v0.1.6-unbalanced/narrative/9936.txt  
  inflating: corpus-ttbr-v0.1.6-unbalanced/

In [5]:
import numpy as np
import pandas as pd
import os

The chunk tags and the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only if two phrases of the same type immediately follow each other, the first word of the second phrase will have tag B-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase. Here is an example:

https://www.clips.uantwerpen.be/conll2003/ner/

In [6]:
import re
from nltk.tokenize import regexp
from nltk.tokenize import sent_tokenize


def replace(sent_id, sentence):
 
  
  tokenizer = regexp.RegexpTokenizer('\w+|[.,?]')

  
  tokens = tokenizer.tokenize(sentence)
  sent_data = []
  for i,token in enumerate(tokens):
    try:
      if token not in ['.', ',', '?']:
        sent_data.append([sent_id,'O',token])
      elif token == '.':
        sent_data[-1][1] = 'I-PERIOD' 
        
      elif token == ',':  
        sent_data[-1][1] = 'I-COMMA' 
       
      elif token == '?':
       sent_data[-1][1] = 'I-QUESTION'
    except IndexError:
      continue
  
  return sent_data

In [7]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## TED Talk dataset

In [8]:
!rm -r ./tedtalk2012-03/ptbr/.ipynb_checkpoints
!rm -r ./tedtalk2012/.ipynb_checkpoints
!rm -r ./tedtalk2012/

rm: cannot remove './tedtalk2012-03/ptbr/.ipynb_checkpoints': No such file or directory
rm: cannot remove './tedtalk2012/.ipynb_checkpoints': No such file or directory
rm: cannot remove './tedtalk2012/': No such file or directory


In [15]:
from nltk.tokenize import sent_tokenize, word_tokenize
from itertools import chain
sent_tokenize('Olá Mundo 1. Olá mundo 2')


sents_ = list(chain.from_iterable(map(sent_tokenize, ['Olá Mundo 1. Olá mundo 2']*2)))
words_ = list(chain.from_iterable(map(word_tokenize,sents_)))


words_

['Olá',
 'Mundo',
 '1',
 '.',
 'Olá',
 'mundo',
 '2',
 'Olá',
 'Mundo',
 '1',
 '.',
 'Olá',
 'mundo',
 '2']

In [16]:
from sklearn.model_selection import train_test_split

def split_df(df):
  sents_ids_uniq = df.sentence_id.astype(np.int32).unique()
  
  trainIdx, testIdx = train_test_split(
      sents_ids_uniq, test_size=0.2, random_state=42)

  trainIdx, devIdx = train_test_split(
      testIdx, test_size=0.2, random_state=42)
  
  sents_id = df['sentence_id'].astype(np.int32).to_numpy()

  train_df = df[np.isin(sents_id, trainIdx )]
  test_df = df[np.isin(sents_id, testIdx )]
  dev_df = df[np.isin(sents_id, devIdx )]

  return train_df, dev_df, test_df

In [17]:
BASE_DIR = './tedtalk2012-03/ptbr/'

PATH_TO_SAVE = './tedtalk2012'

try:
  os.mkdir(PATH_TO_SAVE)
except FileExistsError:
  pass

dict_tedtalk = {
    'dev2010':'dev',
    'train':'train',
    'tst2010':'test'
}


dfs = []
text_dts = []
for filename in os.listdir(BASE_DIR):

  file = open(os.path.join(BASE_DIR, filename), encoding='utf-8')
  data = file.readlines()
  dataset = []
  i = 0
  
  for line in data:
    text = re.sub(r'[!;]', '.', line)
    text = re.sub(r'[:]', ',', text)
    text = re.sub(r'\s[-]\s',',',text)

    emotions = re.findall('\(\w+\)', text)
    if len(emotions) > 0:
      continue
    text_dts.append(text)
    dataset.extend(replace(i,text))
    i += 1
   
  df = pd.DataFrame(np.array(dataset), columns=['sentence_id', 'labels', 'words'])
  dfs.append(df)
  
   
    

In [18]:
tedtalk = pd.concat(dfs)
tedtalk

,sentence_id,labels,words
0,0,O,Pode
1,0,O,ser
2,0,O,algo
3,0,O,bem
4,0,I-COMMA,complicado
...,...,...,...
16255,886,O,poder
16256,886,O,conversar
16257,886,O,com
16258,886,O,todos


In [19]:


sents_ = list(chain.from_iterable(map(sent_tokenize, text_dts)))


info_dict = {
    'tedtalk2012': {
        'num_texts':len(text_dts),
        'num_sents':len(sents_),
        'num_words':tedtalk.shape[0]
    },
    'obras': {
        'num_texts':0,
        'num_sents':0,
        'num_words':0
    },
    'news': {
        'num_texts':0,
        'num_sents':0,
        'num_words':0
    }
}


In [20]:
df_tedtalk = pd.concat(dfs)
tedtalk_train_df,tedtalk_dev_df,tedtalk_test_df =  split_df(df_tedtalk)
tedtalk_train_df.to_csv(f'{PATH_TO_SAVE}/train.csv', index=False, index_label=False)
tedtalk_dev_df.to_csv(f'{PATH_TO_SAVE}/dev.csv', index=False, index_label=False)
tedtalk_test_df.to_csv(f'{PATH_TO_SAVE}/test.csv', index=False, index_label=False)

In [21]:
!zip punct-tedtalk2012.zip -r tedtalk2012/

  adding: tedtalk2012/ (stored 0%)
  adding: tedtalk2012/dev.csv (deflated 72%)
  adding: tedtalk2012/test.csv (deflated 73%)
  adding: tedtalk2012/train.csv (deflated 72%)


In [22]:
!cp punct-tedtalk2012.zip '/content/drive/Shareddrives/MEC - Correção textual/PLN/Dataset/Pontuação/punct-tedtalk2012.zip'

## Obras Dataset

In [23]:
BASE_DIR = 'corpus-ttbr-v0.1.6-unbalanced/narrative'

i = 0
dataset2 = []
text_dts_obras = []
for filename in os.listdir(BASE_DIR):
  file = open(os.path.join(BASE_DIR, filename))
  data = file.readlines()

  for line in data:
    
      text = re.sub(r'[!;]', '.', line)
      text = re.sub(r'[:]', ',', text)
      text = re.sub(r'\s[-]\s',',',text)

      emotions = re.findall('\(\w+\)', text)
      if len(emotions) > 0:
        continue
      text_dts_obras.append(text)
      dataset2.extend(replace(i,text))
      i += 1

In [24]:
obras_df = pd.DataFrame(np.array(dataset2), columns=['sentence_id', 'labels', 'words'])
sents_ = list(chain.from_iterable(map(sent_tokenize, text_dts)))
info_dict['obras']['num_sents'] = len(sents_)
info_dict['obras']['num_texts'] = len(text_dts_obras)
info_dict['obras']['num_words'] = obras_df.shape[0]

In [25]:
obras_train_df, obras_dev_df, obras_test_df = split_df(obras_df)

In [26]:
obras_train_df.shape, obras_test_df.shape, obras_dev_df.shape

((452646, 3), (563945, 3), (111299, 3))

In [27]:
!rm -r obras/

rm: cannot remove 'obras/': No such file or directory


In [28]:

PATH_TO_SAVE = 'obras/'
try:
  os.mkdir(PATH_TO_SAVE)
except FileExistsError:
  pass

In [29]:
obras_train_df.to_csv(f'{PATH_TO_SAVE}/train.csv', index=False, index_label=False)
obras_dev_df.to_csv(f'{PATH_TO_SAVE}/dev.csv', index=False, index_label=False)
obras_test_df.to_csv(f'{PATH_TO_SAVE}/test.csv', index=False, index_label=False)

In [30]:
!zip punct-obras.zip -r obras/

  adding: obras/ (stored 0%)
  adding: obras/dev.csv (deflated 72%)
  adding: obras/test.csv (deflated 72%)
  adding: obras/train.csv (deflated 72%)


In [31]:
!cp punct-obras.zip '/content/drive/Shareddrives/MEC - Correção textual/PLN/Dataset/Pontuação/punct-obras.zip'

## News Dataset

In [32]:
!wget https://github.com/mauricio-seiji/Dataset-news-articles-pt-br/archive/main.zip
!unzip -qq -o main.zip

--2022-05-25 22:27:34--  https://github.com/mauricio-seiji/Dataset-news-articles-pt-br/archive/main.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/mauricio-seiji/Dataset-news-articles-pt-br/zip/refs/heads/main [following]
--2022-05-25 22:27:34--  https://codeload.github.com/mauricio-seiji/Dataset-news-articles-pt-br/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ] 797.33K  --.-KB/s    in 0.09s   

2022-05-25 22:27:34 (9.05 MB/s) - ‘main.zip’ saved [816468]



In [33]:
i = 0
dataset3 = []
news_texts = []
BASE_DIR = './Dataset-news-articles-pt-br-main/'
for dir in os.listdir(BASE_DIR):
  subdir = os.path.join(BASE_DIR, dir)
  if os.path.isdir(subdir):
    for filename in os.listdir(subdir):
        file = open(os.path.join(subdir, filename))
        data = file.readlines()

        for line in data:
          
            text = re.sub(r'[!;]', '.', line)
            text = re.sub(r'[:]', ',', text)
            text = re.sub(r'\s[-]\s',',',text)

            emotions = re.findall('\(\w+\)', text)
            if len(emotions) > 0:
              continue
            news_texts.append(text)
            dataset3.extend(replace(i,text))
            i += 1

In [34]:
news_df = pd.DataFrame(np.array(dataset3), columns=['sentence_id', 'labels', 'words'])
news_df

,sentence_id,labels,words
0,0,O,Propaganda
1,0,O,de
2,0,O,Maguito
3,0,O,em
4,0,O,GO
...,...,...,...
215357,12963,O,a
215358,12963,I-PERIOD,4
215359,12963,I-COMMA,367
215360,12963,O,43


In [35]:
news_train_df, news_dev_df, news_test_df = split_df(news_df)
news_train_df.shape, news_dev_df.shape, news_test_df.shape

((34325, 3), (8449, 3), (42774, 3))

In [36]:
!rm -r news

rm: cannot remove 'news': No such file or directory


In [37]:

PATH_TO_SAVE = 'news/'
try:
  os.mkdir(PATH_TO_SAVE)
except FileExistsError:
  pass

In [38]:
news_train_df.to_csv(f'{PATH_TO_SAVE}/train.csv', index=False, index_label=False)
news_dev_df.to_csv(f'{PATH_TO_SAVE}/dev.csv', index=False, index_label=False)
news_test_df.to_csv(f'{PATH_TO_SAVE}/test.csv', index=False, index_label=False)

In [39]:
sents_ = list(chain.from_iterable(map(sent_tokenize, news_texts)))

info_dict['news'] = {
        'num_texts':len(news_texts),
        'num_sents':len(sents_),
        'num_words':news_df.shape[0]
    }

In [40]:
pd.DataFrame(info_dict).to_latex()

'\\begin{tabular}{lrrr}\n\\toprule\n{} &  tedtalk2012 &    obras &    news \\\\\n\\midrule\nnum\\_texts &       142117 &   254781 &   12964 \\\\\nnum\\_sents &       163718 &   163718 &   11001 \\\\\nnum\\_words &      2300842 &  2820188 &  215362 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [41]:
!zip punct-news.zip -r news/

  adding: news/ (stored 0%)
  adding: news/dev.csv (deflated 71%)
  adding: news/test.csv (deflated 71%)
  adding: news/train.csv (deflated 71%)


In [42]:
!cp punct-news.zip '/content/drive/Shareddrives/MEC - Correção textual/PLN/Dataset/Pontuação/punct-news.zip'

In [43]:
datasets = ['obras', 'news', 'tedtalk2012']
PATH_TO_SAVE = 'punct-data/'

try:
  os.mkdir(PATH_TO_SAVE)
except FileExistsError:
  pass


for dtype in ['train', 'dev', 'test']:
  dfs = []
  for dt in datasets:
     print(os.path.join(dtype, dt))
     dfs.append(pd.read_csv(os.path.join(dt,dtype+'.csv' )))

  new_df = pd.concat(dfs)
  new_df.to_csv(os.path.join(PATH_TO_SAVE, dtype)+'.csv', index=False, index_label=False)


train/obras
train/news
train/tedtalk2012
dev/obras
dev/news
dev/tedtalk2012
test/obras
test/news
test/tedtalk2012


In [44]:
!zip punct-data.zip -r punct-data/

  adding: punct-data/ (stored 0%)
  adding: punct-data/dev.csv (deflated 72%)
  adding: punct-data/test.csv (deflated 72%)
  adding: punct-data/train.csv (deflated 72%)


In [45]:
!cp punct-data.zip '/content/drive/Shareddrives/MEC - Correção textual/PLN/Dataset/Pontuação/punct-data.zip'